# *Procedure* for building " *The df_ib DataFrame from ib770 & ib970 combined with meda_data*

# Import Subroutines and Settings

In [44]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [45]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [46]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


# Startup only Create  **df_ib_fls_raw**  ie  *[COL_NMS = meta+ib770dat + ib970dat]*

## import 770 col nms and 970 col nms

## Calculate the meta column Names from 770 and 970.

## Delete the non data columns from 770 and 970.

## Calculate the 770 column names

## Then calculate 970 add columns names [970 cols - 770 cols]

## Calculate the *df_ib_fls_raw* column names from the beta column names added to the 770 column names to the 970 add column names

## Create *df_ib_fls_raw* using the *df_ib_fls_raw* column names.

## Then save it to pickle.

# Import the ib770 & ib970 to **df_ib_fls_raw**   ok

In [48]:
ib_fls_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib_fls",
    pattern="*"
)
df_ib_fls_raw = pd.DataFrame(ib_fls_raw)

# verify
# 
print( df_ib_fls_raw["2. ID"])


✅ Loaded 251201-1_20251207201219.csv with ISO-8859-1
✅ Loaded 091725-1_20251118042051.csv with ISO-8859-1
✅ Loaded 091725-1_20251023065307.csv with ISO-8859-1
✅ Loaded 091725-1_20250928235640.csv with ISO-8859-1
✅ Loaded 091725_20251213054921.csv with utf-8
✅ Loaded 091725-1_20251025233431.csv with ISO-8859-1
✅ Loaded 091725_20251214234814.csv with utf-8
✅ Loaded 091725-1_20251124075021.csv with ISO-8859-1
✅ Loaded 091725-1_20251021000847.csv with ISO-8859-1
✅ Loaded 091725-1_20251101213555.csv with ISO-8859-1
✅ Loaded 251201-1_20251205081353.csv with ISO-8859-1
✅ Loaded 091725-1_20251116230333.csv with ISO-8859-1
✅ Loaded 251201-1_20251206072118.csv with ISO-8859-1
✅ Loaded 091725-1_20251130055027.csv with ISO-8859-1
✅ Loaded 091725-1_20251019230750.csv with ISO-8859-1
✅ Loaded 091725-1_20251017091340.csv with ISO-8859-1
✅ Loaded 091725-1_20251002065435.csv with ISO-8859-1
✅ Loaded 091725-1_20251019074606.csv with ISO-8859-1
✅ Loaded 091725-1_20251118230931.csv with ISO-8859-1
✅ Loade

In [56]:
# verify
# print( df_ib_fls_raw["2. ID"])

# df_col_nm_lst =["",""]
df_col_nm_lst = pd.DataFrame( df_ib_fls_raw.columns)
# verify
print(df_col_nm_lst)

                                                     0
0                                              1. Name
1                                                2. ID
2                                            3. Height
3                                     4. Date of Birth
4                                               5. Age
5                                     6. Mobile Number
6                                      7. Phone Number
7                                          8. Zip Code
8                                           9. Address
9                                           10. E-mail
10                            11. Date of Registration
11                                            12. Memo
12                                13. Test Date / Time
13                                          14. Weight
14               15. Lower Limit (Weight Normal Range)
15               16. Upper Limit (Weight Normal Range)
16                          17. TBW (Total Body Water)
17        

# Remove **numbers** from col_nms   ok

In [63]:
# STRIPS COL NMBRS  column #s from col heads

def strip_col_numbers(col_names):
    """
    Remove leading numbers and dots from column names.
    Example: '244. 50kHz-Whole Body Phase Angle_Z score' 
             -> '50kHz-Whole Body Phase Angle_Z score'
    """
    return [re.sub(r'^\d+\.\s*', '', name) for name in col_names]

# Run this for both the 970 and the 770 data imports
col_names = C.columns
df_ib_fls_raw_nm = pd.DataFrame(strip_col_numbers(col_names))
# verify
# print(df_ib_fls_raw_nm)

# Remove Duplicates so each col in 770 and 970 Have one unique Column In this overall *df_ib* dataframe   OK

In [65]:
df_ib_fls_raw_nm = df_ib_fls_raw_nm.drop_duplicates(keep="first").reset_index(drop=True)


In [66]:
# verify
# 
print(df_ib_fls_raw_nm)

                                                     0
0                                                 Name
1                                                   ID
2                                               Height
3                                        Date of Birth
4                                                  Age
5                                        Mobile Number
6                                         Phone Number
7                                             Zip Code
8                                              Address
9                                               E-mail
10                                Date of Registration
11                                                Memo
12                                    Test Date / Time
13                                              Weight
14                   Lower Limit (Weight Normal Range)
15                   Upper Limit (Weight Normal Range)
16                              TBW (Total Body Water)
17        

In [70]:
df_ib_fls_raw

,1. Name,2. ID,3. Height,4. Date of Birth,5. Age,6. Mobile Number,7. Phone Number,8. Zip Code,9. Address,10. E-mail,...,248. HGS of Left Arm 1st,249. HGS of Left Arm 2nd,250. HGS of Right Arm 1st,251. HGS of Right Arm 2nd,252. HGS/WT,Unnamed: 252,152 50KH,-33.5,jb,bjh970
0,251201-1,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,bjh970,091725-1,6ft 06.0in,1939.02.23.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
